In [45]:
import pandas as pd

df = pd.read_csv("Practical Example - Confidence Intervals.xlsx - Al Bundy.csv",skiprows=3, header=0)
df.set_index("InvoiceNo", inplace = True)

df = df.drop(columns = ["Unnamed: 0", "SalePrice", "Discount", "UnitPrice", "Size (Europe)", "Size (UK)", "Date"])
df.rename(columns = {"Unnamed: 12" : "Year"}, inplace=True)

df

,Country,ProductID,Shop,Gender,Size (US),Year,Month
InvoiceNo,,,,,,,
52389,United Kingdom,2152,UK2,Male,11.0,2014,1
52390,United States,2230,US15,Male,11.5,2014,1
52391,Canada,2160,CAN7,Male,9.5,2014,1
52392,United States,2234,US6,Female,9.5,2014,1
52393,United Kingdom,2222,UK4,Female,9.0,2014,1
...,...,...,...,...,...,...,...
65773,United Kingdom,2154,UK2,Male,9.5,2016,12
65774,United States,2181,US12,Female,12.0,2016,12
65775,Canada,2203,CAN6,Male,10.5,2016,12


In [57]:
men2016_df = df.loc[(df['Year'] == 2016) & (df['Gender'] == 'Male')]
fem2016_df = df.loc[(df['Year'] == 2016) & (df['Gender'] == 'Female')]

m_can = men2016_df["Country"] == "Canada"
m_us = men2016_df["Country"] == "United States"
m_uk = men2016_df["Country"] == "United Kingdom"
m_ger = men2016_df["Country"] == "Germany"


men2016_df[m_can]["Size (US)"].value_counts()

10.0    117
11.0     59
10.5    114
7.5      22
9.0     145
13.0      6
8.0      28
8.5      87
9.5     159
14.0     12
11.5     43
12.0     22
7.0      12
15.0      8
6.5       8
6.0       6
Name: Size (US), dtype: int64

In [59]:
df["Country"].unique()

array(['United Kingdom', 'United States', 'Canada', 'Germany'],
      dtype=object)